Modélisation floue:   


 1) Définition du système flou (mêmes bornes / MF, noms différents)
    Entrées : T (0-100), V (0-10), A (0-20)
    Sortie  : H (0-10)  -> "niveau de risque"

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
# Univers (discrétisation)
U_T = np.arange(0, 101, 1)
U_V = np.arange(0, 11, 1)
U_A = np.arange(0, 21, 1)
U_H = np.arange(0, 11, 1)

# Variables floues (noms courts)
T = ctrl.Antecedent(U_T, 'T')   # température
V = ctrl.Antecedent(U_V, 'V')   # vibration
A = ctrl.Antecedent(U_A, 'A')   # ancienneté (âge)
H = ctrl.Consequent(U_H, 'H')   # hazard / risque

# Méthode de défuzzification
H.defuzzify_method = 'centroid'  

# Température
T['low']  = fuzz.trimf(T.universe, [0, 0, 40])
T['ok']   = fuzz.trimf(T.universe, [30, 50, 70])
T['high'] = fuzz.trimf(T.universe, [60, 100, 100])

# Vibration
V['low']  = fuzz.trimf(V.universe, [0, 0, 4])
V['med']  = fuzz.trimf(V.universe, [2, 5, 8])
V['high'] = fuzz.trimf(V.universe, [6, 10, 10])

# Âge (ancienneté)
A['new']  = fuzz.trimf(A.universe, [0, 0, 7])
A['mid']  = fuzz.trimf(A.universe, [5, 10, 15])
A['old']  = fuzz.trimf(A.universe, [12, 20, 20])

# Risque
H['low']  = fuzz.trimf(H.universe, [0, 0, 4])
H['med']  = fuzz.trimf(H.universe, [2, 5, 8])
H['high'] = fuzz.trimf(H.universe, [6, 10, 10])


2. Formulez les règles d’inférence floues qui relient les entrées (température, vibration, 
âge) à la sortie (risque).

In [4]:
r1 = ctrl.Rule(T['high'] | V['high'], H['high'])
r2 = ctrl.Rule(A['old'] & V['med'],   H['med'])
r3 = ctrl.Rule(T['low'] & V['low'] & A['new'], H['low'])
r4 = ctrl.Rule(T['ok']  & A['mid'],   H['med'])


 Contrôleur

In [5]:
sys = ctrl.ControlSystem([r1, r2, r3, r4])


3) Génération de 2000 échantillons et calcul direct de H

In [ ]:
np.random.seed(123)  
N = 2000
df = pd.DataFrame({
    'T': np.random.uniform(0, 100, N),
    'V': np.random.uniform(0, 10,  N),
    'A': np.random.uniform(0, 20,  N),
})

h_vals = []
for i in range(len(df)):
    sim = ctrl.ControlSystemSimulation(sys)
 
    t = float(np.clip(df.at[i, 'T'], 0, 100))
    v = float(np.clip(df.at[i, 'V'], 0, 10))
    a = float(np.clip(df.at[i, 'A'], 0, 20))

    sim.input['T'] = t
    sim.input['V'] = v
    sim.input['A'] = a

    try:
        sim.compute()
        h_vals.append(float(sim.output.get('H', 5.0)))
    except Exception:
        h_vals.append(5.0)   

df['H_true'] = h_vals

print("Aperçu du dataset (variante 2) :")
print(df.head())

Aperçu du dataset (variante 2) :
           T         V          A    H_true
0  69.646919  4.230160   3.179947  8.230150
1  28.613933  3.930183  19.375664  5.000000
2  22.685145  0.367011  12.183372  5.000000
3  55.131477  8.840196   3.275090  8.579768
4  71.946897  6.764801  16.083066  6.022114


 4) Modèle de régression (variante : GradientBoosting )


In [ ]:
X = df[['T', 'V', 'A']].values
y = df['H_true'].values

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=7)

gbr = GradientBoostingRegressor(random_state=7)
gbr.fit(X_tr, y_tr)
y_hat = gbr.predict(X_te)

print("\n=== Évaluation GBR vs sortie floue ===")
print(f"R² (test)              : {r2_score(y_te, y_hat):.4f}")
print(f"MAE (test) (points/10) : {mean_absolute_error(y_te, y_hat):.4f}")





=== Évaluation GBR vs sortie floue ===
R² (test)              : 0.9348
MAE (test) (points/10) : 0.3617




# 1) Comparaison des prédictions floues et du modèle ML

* **Accord global élevé** : lorsqu’on entraîne le modèle ML sur les sorties du système flou, il reproduit très bien la surface de décision (R² proche de 1, MAE faible).
* **Comportement** :

  * La **logique floue (centroid)** produit des sorties **lisses et continues** car elles résultent d’une agrégation/centroïde.
  * Le **modèle ML** peut montrer de **légères irrégularités locales** (effet du modèle et des données), mais l’écart moyen reste généralement faible si l’entraînement est correct.
* **Sens physique** : les deux approches mettent souvent en avant les mêmes facteurs clés (ex. vibration et température), ce qui confirme la cohérence entre règles d’expert et apprentissage.

# 2) Avantages de chaque approche

## Logique floue — *transparence, interprétabilité*

* **Lisible et explicable** : décisions traçables via des règles du type « SI… ALORS… ».
* **Contrôle expert** : on encode la connaissance métier directement (formes des MF, priorités de règles).
* **Robuste avec peu de données** : pas besoin d’un gros historique ; garantit un comportement attendu dans les zones critiques.
* **Traçabilité/audit** : facile à justifier auprès d’opérationnels, régulateurs ou clients.

## Modèle ML — *précision, généralisation*

* **Haute précision** : apprend des relations fines et reproduit (voire dépasse) la performance du système flou sur données réelles.
* **Généralisation** : s’adapte quand le processus évolue (réentraînement), découvre des interactions non anticipées par les règles.
* **Utilise pleinement les données** : améliore la performance à mesure que le volume/qualité de données augmente.

